# Deep Learning Project
## Istanbul Technical University
### Computer Engineering Department
### BLG 527E - Deep Learning
### Spring 2023

<b>Student Name: Muhammed Rüşen Birben</b><br>
<b>Student ID: 150220755</b><br>
<b>Student Email: birben20@itu.edu.tr</b><br>

<b>Student Name: Ahmed Burak Ercan</b><br>
<b>Student ID: 150220749</b><br>
<b>Student Email: ercana20@itu.edu.tr</b><br>

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from os import makedirs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from utils import get_model, model_predict, get_label_output
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import importlib
# To reload the module after modifications
"""importlib.reload(get_model)
importlib.reload(model_predict)
importlib.reload(get_label_output)"""

/Users/rusenbirben/anaconda3/envs/deep_learning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'importlib.reload(get_model)\nimportlib.reload(model_predict)\nimportlib.reload(get_label_output)'

In [2]:
# import warnings
# warnings.filterwarnings("ignore")

### Loading the Data

In [3]:
# Reading the data.
df = pd.read_csv('datasetV2.csv')
df.head()
#df = df[:100]

,Source,Text,IsAI
0,LLM,Have you ever heard of the Crusades? A time in...,1
1,LLM,"The professors, who likely have nearly a decad...",1
2,LLM,Kemba Walker does a good job of defending Foye...,1
3,LLM,"Ganias' lawyer, Stanley Twardy, urged the gove...",1
4,LLM,The Circuit Court of Appeals of New Jersey had...,0


In [4]:
df_filtered = df[df["IsAI"] == 0][:500]
df_filtered = pd.concat([df_filtered, df[df["IsAI"] == 1][:500]]).reset_index(drop=True)

In [5]:
X = df_filtered.Text
y = df_filtered.IsAI

### Model Development

In [6]:
def predict(model_name, X):
    # load model
    model, tokenizer = get_model(model_name)

    # sentences
    text = X.values.tolist()

    # predict
    predictions = model_predict(model, tokenizer, text)

    return predictions, model, tokenizer

In [7]:
def eval_model(y_predictons, y_true):
    #Evaluate the model by computing precision, recall and F1-score.
    pred_labels = get_label_output(y_predictons)

    accuracy = accuracy_score(y_true, pred_labels)
    recall = recall_score(y_true, pred_labels, average='macro')
    precision = precision_score(y_true, pred_labels, average='macro')
    f1 = f1_score(y_true, pred_labels, average='macro')
    
    conf_matrix = confusion_matrix(y_true, pred_labels)
    
    print('Accuracy: {:.2f}%'.format(accuracy*100))
    print('Recall: {:.2f}%'.format(recall*100))
    print('Precision: {:.2f}%'.format(precision*100))
    print('F1-score: {:.2f}%'.format(f1*100))

    return conf_matrix

In [8]:
predictions, model, tokenizer = predict("roberta-base-openai-detector", X)

Model already exists, loading...


100%|██████████| 10/10 [00:43<00:00,  4.35s/it]


In [9]:
eval_model(predictions, y)

Accuracy: 65.60%
Recall: 65.60%
Precision: 65.80%
F1-score: 65.49%


/Users/rusenbirben/anaconda3/envs/deep_learning/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/rusenbirben/anaconda3/envs/deep_learning/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/rusenbirben/anaconda3/envs/deep_learning/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/rusenbirben/anaconda3/envs/deep_learning/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will

array([[356, 144],
       [200, 300]])

### Fine-Tuning

In [10]:
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
# Splitting the DataFrame into train and remaining datasets
# get a portion of the dataset
df = df.sample(frac=0.1, random_state=42)
df = df[:1000] # might increase this later
train_df, remaining_df = train_test_split(df, test_size=0.30, random_state=42)

# Splitting the remaining dataset into validation and test datasets
val_df, test_df = train_test_split(remaining_df, test_size=0.4, random_state=42)


train_encodings = tokenizer(train_df['Text'].tolist(), padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(val_df['Text'].tolist(), padding=True, truncation=True, max_length=512)
test_encodings = tokenizer(test_df['Text'].tolist(), padding=True, truncation=True, max_length=512)

train_dataset = TextDataset(train_encodings, train_df['IsAI'].tolist())
val_dataset = TextDataset(val_encodings, val_df['IsAI'].tolist())
test_dataset = TextDataset(test_encodings, test_df['IsAI'].tolist())

In [12]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=2,   # batch size per device during training
    per_device_eval_batch_size=2,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.02,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    evaluation_strategy="epoch"      # evaluate each `logging_steps`
)

# Initialize Trainer
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

# Train and evaluate
trainer.train()

  1%|          | 10/1050 [00:03<05:46,  3.00it/s]

{'loss': 0.9515, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.03}


  2%|▏         | 20/1050 [00:06<05:11,  3.30it/s]

{'loss': 0.698, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.06}


  3%|▎         | 30/1050 [00:09<05:04,  3.35it/s]

...

100%|██████████| 1050/1050 [06:06<00:00,  2.91it/s]

{'loss': 0.0003, 'learning_rate': 0.0, 'epoch': 3.0}


                                                   
100%|██████████| 1050/1050 [06:12<00:00,  2.82it/s]

{'eval_loss': 0.4705905616283417, 'eval_runtime': 5.5831, 'eval_samples_per_second': 32.24, 'eval_steps_per_second': 16.12, 'epoch': 3.0}
{'train_runtime': 372.396, 'train_samples_per_second': 5.639, 'train_steps_per_second': 2.82, 'train_loss': 0.3610114858565586, 'epoch': 3.0}


TrainOutput(global_step=1050, training_loss=0.3610114858565586, metrics={'train_runtime': 372.396, 'train_samples_per_second': 5.639, 'train_steps_per_second': 2.82, 'train_loss': 0.3610114858565586, 'epoch': 3.0})

In [13]:
predictions = trainer.predict(test_dataset)

100%|██████████| 60/60 [00:03<00:00, 19.27it/s]


In [14]:
# softmax
predictons = F.softmax(torch.from_numpy(predictions.predictions), dim=1)
predictons = predictons.detach().numpy()


In [15]:
# argmax label
pred_labels = get_label_output(predictons)
y = test_df['IsAI'].tolist()

In [16]:
#eval_model(pred_labels, y) # y düzenle

accuracy = accuracy_score(y, pred_labels)
recall = recall_score(y, pred_labels, average='macro')
precision = precision_score(y, pred_labels, average='macro')
f1 = f1_score(y, pred_labels, average='macro')

conf_matrix = confusion_matrix(y, pred_labels)

print('Accuracy: {:.2f}%'.format(accuracy*100))
print('Recall: {:.2f}%'.format(recall*100))
print('Precision: {:.2f}%'.format(precision*100))
print('F1-score: {:.2f}%'.format(f1*100))


Accuracy: 91.67%
Recall: 91.75%
Precision: 91.56%
F1-score: 91.63%
